In [1]:
class Peptide:
    def __init__(self):
        self.masses = []
    
    @property
    def mass(self):
        return sum(self.masses) 
        
    @property
    def spectrum(self):
        sp = [0]
        mses = self.masses.copy()[:-1]
        n = len(self.masses)
        for k in range(1, n):
            mses.append(self.masses[(k-2)%n])
            for i in range(len(mses)-k+1):
                sp.append(sum(mses[i:i+k]))
        sp.append(sum(self.masses))
        sp.sort()
        return sp
    
    def add(self, new_mass):
        self.masses.append(new_mass)
        return self
        
    def __str__(self):
        string = ''
        for m in self.masses:
            string += str(m) + '-'
        string = string[:-1]
        return string
    
    def copy(self):
        copy_pep = Peptide()
        copy_pep.masses = self.masses.copy()
        return copy_pep

In [2]:
from collections import Counter
def top_m_elements(spectrum, M):
    masses = [57, 71, 87, 97, 99, 101, 103, 113, 114, 115, 128, 129, 131, 137, 147, 156, 163, 186]
    differences = []
    for first_elem in spectrum:
        for second_elem in spectrum:
            diff = abs(first_elem - second_elem)
            if 57 <= diff and diff <= 200:
                differences.append(diff)
    c = Counter(differences)
    new_min_count = c.most_common(M)[-1][1]
    new_masses = [elem for elem in c if c[elem] >= new_min_count and elem in masses]
    new_masses.sort()
    return new_masses

In [3]:
def expand(peptides, masses):
    new_peptides = [peptide.copy().add(m) for m in masses for peptide in peptides]
    return new_peptides

In [4]:
def score(peptide, spectrum):
    sc = 0
    spec = spectrum.copy()
    for m in peptide.spectrum:
        if m in spec:
            sc += 1
            spec.remove(m)
    return sc

In [5]:
def cut(leaderboard, spectrum, N):
    if len(leaderboard) == 0: 
        return []
    scores = [score(peptide, spectrum) for peptide in leaderboard]
    scores.sort(reverse = True)
    top_score = scores[min(N, len(scores))-1]
    new_leaderboard = [peptide for peptide in leaderboard if score(peptide,spectrum) >= top_score]
    return new_leaderboard

In [6]:
def ConvolutionCyclopeptideSequencing(spectrum, N, M):
    masses = top_m_elements(spectrum, M)
    leaderboard = [Peptide()]
    leaderPeptide = Peptide()
    while len(leaderboard) > 0:
        leaderboard = expand(leaderboard, masses)
        new_leaderboard  = leaderboard.copy()
        for peptide in leaderboard:
            if peptide.mass == max(spectrum):
                if score(peptide, spectrum) > score(leaderPeptide, spectrum):
                    leaderPeptide = peptide.copy()
            elif peptide.mass > max(spectrum):
                new_leaderboard.remove(peptide)
        new_leaderboard = cut(new_leaderboard, spectrum, N)                     
        leaderboard = new_leaderboard
    return leaderPeptide

In [7]:
filename = 'rosalind_ba4i.txt'

In [8]:
with open(filename) as file:
    M = int(file.readline().rstrip())
    N = int(file.readline().rstrip())
    spectrum = [int(x) for x in file.readline().split()]

In [9]:
spectrum.sort()
leaderPeptide = ConvolutionCyclopeptideSequencing(spectrum, N, M)
print(leaderPeptide)

156-114-163-71-71-99-128-163-147-128-156-113-113-99-57
